# Algoritmia
## Práctica Obligatoria 3
### Curso 2023 - 2024
###### Programación dinámica
---
 

#### Autores:
* Jimena Arnaiz González
* Iván Estépar Rebollo

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar librerías nativas (https://docs.python.org/es/3.8/library/index.html)
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

In [1]:
#testeable
# Imports útiles


In [2]:
#testeable
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """    
    
    def __init__(self, name, size):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size
        
        self.dicc_users = {}
    
    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """        
        return hash(self.name)
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """        
        return f"Vídeo {self.name} de tamaño {self.size}" 
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """  
        return self.__str__()
       
    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """        
        #Si el país ya está en el diccionario, actualiza la cantidad de usuarios
        if country in self.dicc_users:
            self.dicc_users[country] += users
        #si no, se agrega una nueva entrada
        else:
            self.dicc_users[country] = users
    
    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """        
        return self.dicc_users.get(country, 0)

    def tiempo_emision(self, country):
        """Dado un país, obtiene el tiempo de emisión.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de minutos de emisión para el país `country`
        """
             
        # Calculamos el tiempo de emisión multiplicando el tamaño del video por el número de usuarios
        tiempo_emision = self.size * self.get_users(country)
        
        return tiempo_emision


In [3]:
#testeable
class ServidorCache:
    """
    Clase del servidor caché donde se almacenan parte de series/películas.
    """
    
    def __init__(self, identifier, country, capacity):
        """Instancia un Servidor de Caché

        Parameters
        ----------
        identifier : int
            Valor que identifica un servidor.
        country : str
            País donde está el servidor.
        capacity : int
            Cantidad de memoria de almacenamiento disponible.
        """
        self.identifier = identifier
        self.country = country
        self.capacity = capacity
        
        self.videosAlmac = []
        
    def __hash__(self):
        """Genera el valor hash identificativo del servidor

        Returns
        -------
        int
            Valor hash
        """    
        return hash(self.identifier)

    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f"Servidor {self.identifier} en {self.country} con capacidad {self.capacity}"
    
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return self.__str__()
    
    def rellena(self, videos):
        """Dada una colección de videos,
           seleccionar aquellos que se van a almacenar en el servidor.
           Se ha de optimizar para que el tiempo de emisión sea el máximo posible.
           No se pueden partir los vídeos.

        Parameters
        ----------
        videos : collection
            Colección de videos que se quieren almacenar en el servidor.
        """
        # Creamos una matriz para almacenar los valores máximos
        n = len(videos)
        W = self.capacity
        V = [[0 for _ in range(W + 1)] for _ in range(n + 1)]
        
        # Rellenamos la tabla V usando el algoritmo de la mochila con programación dinámica
        for i in range(1, n + 1):
            for j in range(1, W + 1):
                weight = videos[i - 1].size
                value = videos[i - 1].tiempo_emision(self.country)
                if weight <= j:
                    V[i][j] = max(V[i - 1][j], V[i - 1][j - weight] + value)
                else:
                    V[i][j] = V[i - 1][j]
        
        # Recuperamos la solución óptima
        j = W
        selected_videos = []
        for i in range(n, 0, -1):
            if V[i][j] != V[i - 1][j]:
                selected_videos.append(videos[i - 1])
                j -= videos[i - 1].size
        
        self.videosAlmac = selected_videos
        
                    
    def tiempo_emision(self):
        """A partir de los datos almacenados
           devolver el tiempo de emisión óptimo del servidor.

        Returns
        -------
        number
            Tiempo de Emision            
        """        
        tiempo_emision_total = 0
    
        # Itera sobre los videos
        for video in self.videosAlmac:
            tiempo_emision_total += video.tiempo_emision(self.country)

        return tiempo_emision_total
    

    def almacenados(self):
        """A partir de los datos almacenados
           devolver los objetos vídeo.

        Returns
        -------
        collection
            Colección de videos almacenados en el servidor.
        """     
        
        return self.videosAlmac
    
    
    def __lt__(self, other):
        return self.identifier < other.identifier
    
    
    def __le__(self, other):
        return self.identifier <= other.identifier
    

In [4]:
#testeable
class ServidorMaestro:
    """
    Servidor central que gestiona las conexiones entre servidores cache
    """
    
    def __init__(self, servidores, distancias):
        """Instancia el servidor central

        Parameters
        ----------
        servidores : Iterable
            Conjunto de servidores cache disponibles
        distancias : dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """        
        self.servidores = set(servidores)
        self.distancias = distancias   
        self.distancias_minimas = {}
        self.caminos_minimos = {}
          
    def get_grafo(self):
        """Devuelve el grafo de distancias recibido

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """        
        return self.distancias

    def calcula_distancias(self):
        """Calcula las distancias MÍNIMAS entre servidores cache
           y los correspondientes caminos utilizando el algoritmo de Floyd.
        """
        # Inicializa las matrices de distancias mínimas y caminos mínimos
        n = len(self.servidores)
        for servidor in self.servidores:
            self.distancias_minimas[servidor] = {}
            self.caminos_minimos[servidor] = {}
            for otro_servidor in self.servidores:
                self.distancias_minimas[servidor][otro_servidor] = float('inf')
                self.caminos_minimos[servidor][otro_servidor] = []

        # Llena las matrices con las distancias directas y caminos directos
        for servidor1 in self.servidores:
            for servidor2 in self.servidores:
                if servidor2 in self.distancias[servidor1]:
                    self.distancias_minimas[servidor1][servidor2] = self.distancias[servidor1][servidor2]
                    self.caminos_minimos[servidor1][servidor2] = [servidor1, servidor2]

        # Algoritmo de Floyd
        for k in self.servidores:
            for i in self.servidores:
                for j in self.servidores:
                    if self.distancias_minimas[i][j] > self.distancias_minimas[i][k] + self.distancias_minimas[k][j]:
                        self.distancias_minimas[i][j] = self.distancias_minimas[i][k] + self.distancias_minimas[k][j]
                        self.caminos_minimos[i][j] = self.caminos_minimos[i][k] + self.caminos_minimos[k][j][1:]

    def distancia(self, origen, destino):
        """
        Devuelve la distancia mínima entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        int
            Distancia mínima en milisegundos entre los servidores.
        """
        if origen not in self.servidores or destino not in self.servidores:
            raise ValueError("Los servidores deben estar en la red del maestro.")

        if destino not in self.distancias_minimas:
            self.calcula_distancias()

        return self.distancias_minimas[origen][destino]

    def camino(self, origen, destino):
        """
        Devuelve el camino mínimo entre dos servidores cache.

        Parameters
        ----------
        origen : ServidorCache
            Servidor de origen
        destino : ServidorCache
            Servidor de destino

        Returns
        -------
        list<ServidorCache>
            Lista de servidores para llegar de origen a destino. 
            Se debe incluir al origen y al destino.
        """
        if origen not in self.servidores or destino not in self.servidores:
            raise ValueError("Los servidores deben estar en la red del maestro.")
       
        # Si el origen es igual al destino, el camino es solo el nodo y el costo es 0
        if origen == destino:
            return [origen]

        if destino not in self.caminos_minimos:
            self.calcula_distancias()

        camino = self.caminos_minimos[origen][destino]
        
        return camino if camino else []

### Pruebas de ejemplo

In [5]:
import unittest
import json

def carga_dataset(data):
    with open(data) as f:
        test_datasets = json.load(f)

    videos = list()
    for v in test_datasets["videos"]:
        v_obj = Video(v["name"], v["size"])
        for c, u in v["users"].items():
            v_obj.set_users(c, u)
        videos.append(v_obj)

    servers = dict()
    for s in test_datasets["servers"]:
        servers[s["country"]] = ServidorCache(s["identifier"], s["country"], s["size"])


    pings = test_datasets["pings"]
    p_ = dict()
    for s in servers.values():
        p_[s] = dict()
        for p in pings[s.country]:
            ping = pings[s.country][p]
            if pings[s.country][p] == -1:
                ping = float("inf")
            p_[s][servers[p]] = ping
    maestro = ServidorMaestro(servers.values(), p_)

    return videos, servers, maestro

class TestBasico(unittest.TestCase):
    
    def test_carga_simple(self):
        
        v, s, m = carga_dataset("toy_PD.json")

        spain = s["Spain"]
        spain.rellena(v)
        self.assertEqual(spain.tiempo_emision(), 55800)
        almacenados = spain.almacenados()
        self.assertEqual(len(almacenados), 3)

        m.calcula_distancias()
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.camino(s["France"], s["Ireland"]), [s["France"], s["Spain"], s["Ireland"]])
        self.assertEqual(m.distancia(s["France"], s["Ireland"]), 200)      
        
    

In [6]:
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```

Los tests necesitan de la librería `nbformat`

```bash
$ pip install nbformat
```

###### Explicación de los tests
* `test_ej1_tiempo_emision_correcto`: Comprueba que el tiempo de emisión es el máximo para cada servidor.
* `test_ej1_emision_maxima`: Comprueba que ante un servidor virtualmente infinita el tiempo de emisión es el máximo posible.
* `test_ej1_pais_no_existe`: Comprueba que el tiempo de emisión de un país que no existe es 0.
* `test_ej2_distancias_correctas`: Comprueba que la distancia es la mínima posible.
* `test_ej2_caminos_correctos`: Comprueba que los caminos son los mínimos.
* `test_ej2_camino_nulo`: Comprueba que ante un viaje cuyo origen es igual al destino el camino es solo el nodo y el coste es 0.

---

### **Informe**
Contesta a las siguientes preguntas (Justificando las respuestas).

#### **Complejidad**

1. Método `ServidorCache.rellena`
    * **Complejidad temporal**:
    La complejidad temporal de este método se debe a la implementación del algoritmo de la mochila con programación dinámica. En este caso, se usa un enfoque de programación dinámica para encontrar la selección óptima de videos que maximice el tiempo de emisión. Suponiendo que hay n videos y la capacidad del servidor es W, la complejidad temporal es O(n×W), donde n es el número de videos y W es la capacidad del servidor.
2. Método `ServidorMaestro.calcula_distancias`
    * **Complejidad temporal**:
    Este método implementa el algoritmo de Floyd para encontrar los caminos y distancias mínimas entre todos los pares de servidores cache en la red. Suponiendo que hay n servidores en la red, la complejidad temporal del algoritmo de Floyd es O(n^3). Sin embargo, dado que el grafo en cuestión es un grafo completo de servidores (cada servidor está conectado con todos los demás), la complejidad real es O(n^2) en este caso.
    


#### **Respecto a la maximización del tiempo de emisión del servidor caché.**

* ¿Cómo de buena es la solución? ¿Es óptima o aproximada?<br><sub style="font-style: italic">No confundir una solución óptima (obtiene el máximo posible) frente a una solución eficiente (calcula el máximo con la menor complejidad posible).</sub>
</br>

La solución es óptima. El método rellena implementa el algoritmo de la mochila con programación dinámica, que es conocido por proporcionar la solución óptima para este tipo de problemas de optimización con restricciones de capacidad.


* ¿Cómo cambiaría la complejidad espacial si aumentase el tamaño del servidor? ¿Y si aumentase el número de vídeos?

Como la complejidad del algoritmo es O(nxW), donde n es el número de vídeos y W es la capacidad del servidor, si aumenta el tamaño del servidor, la complejidad espacial aumentaría linealmente con respecto a la capacidad del servidor. Por lo tanto, si el tamaño del servidor aumenta, la complejidad espacial aumentará en consecuencia.<br>
De igual forma, si aumenta el número de vídeos, la complejidad espacial aumentaría de manera lineal con respecto al número de vídeos.


#### **Respecto a la búsqueda de distancias mínimas entre servidores.**

* Este problema tiene una aproximación voraz, ¿cuál sería la ventaja de usarlo frente a la solución que has planteado? 

La principal ventaja de utilizar una aproximación voraz, como el algoritmo de Dijkstra, frente a nuestra solución con el algoritmo de Floyd, es su eficiencia en términos de tiempo de ejecución en grafos grandes. Esto hace que el algoritmo de Dijkstra sea más adecuado para grafos grandes, ya que su tiempo de ejecución es más rápido que el algoritmo de Floyd.

* ¿Y cuál sería su desventaja?

La principal desventaja del enfoque voraz, como el algoritmo de Dijkstra, es que no garantiza encontrar la solución óptima en todos los casos. Aunque puede encontrar una solución rápida, no necesariamente encuentra la distancia mínima en todos los casos, especialmente si hay pesos negativos en el grafo. 